In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
%matplotlib inline
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
data = pd.read_csv("../input/movie_metadata.csv")
data.head()

In [ ]:
data['return_rate']= data['gross']/data['budget']
top5_easy =data.groupby('return_rate',sort = False).sum() 
top5_easy.sort_index(ascending=False)
top5_easy.head()

In [ ]:
data.info()

In [ ]:
cor = data.corr()
sns.heatmap(cor)

In [ ]:
data=data.dropna()
data = data.drop(['color','director_name','actor_2_name','genres','actor_1_name','movie_title','actor_3_name','plot_keywords','movie_imdb_link','language','country','content_rating'],axis = 1)

In [ ]:
from sklearn.model_selection import train_test_split
label = np.asarray(data['imdb_score'], dtype="|S6")
data = data.drop('imdb_score',axis =1)

X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.33, random_state=42)

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import f1_score

clf_A = GaussianNB()
clf_B = LogisticRegression(random_state=42)
clf_C = SVC(random_state=42)
l = [clf_A,clf_B,clf_C]
results = []
def choose_model(class_list):
    for clf in class_list:
        clf.fit(X_train, y_train)
        y_pred = clf.predict(data)
        results.append(f1_score(label, y_pred, average = 'micro'))
    return results
    
choose_model(l)
    

        

In [ ]:
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV

def get_estimator():
    C = [0.001, 0.01, 0.1, 1, 10, 100, 1000]

    max_iter = [1000]
    param_grid = dict(C=C, max_iter=max_iter)

    clf = SVC(random_state=42)

    f1_scorer = make_scorer(f1_score, pos_label='yes',average = 'micro')

    grid_obj = GridSearchCV(clf, param_grid, scoring=f1_scorer)
    grid_obj = grid_obj.fit(X_train, y_train)
    clf = grid_obj.best_estimator_
    return clf
get_estimator()
